In [11]:
import sys
import os
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [12]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            3, 6, 5
        )  # (3, 256, 256) -> new dim (256 - 5 + (2 * 0)) / 1 + 1 ) = 252 (6, 252, 252)
        self.pool = nn.MaxPool2d(2, 2)  # (6, 252, 252) -> (6, 126, 126)
        self.conv2 = nn.Conv2d(
            6, 16, 5
        )  # (6, 126, 126) -> (16, 122, 122) -> MaxPool -> (16, 61, 61)
        self.fc1 = nn.Linear(16 * 61 * 61, 2048)
        self.fc2 = nn.Linear(2048, 256)
        self.fc3 = nn.Linear(256, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:

def create_dataloaders(dataset):
    data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    dataiter = iter(data_loader)
    images, labels = next(dataiter)
    # print(images, labels)
    print(dataset.classes)
    # imshow(torchvision.utils.make_grid(images))
    labels = np.array([dataset.targets[i] for i in range(len(dataset))])

    # Define split sizes
    train_size = 0.7  # 70% for training
    val_size = 0.15  # 15% for validation
    test_size = 0.15  # 15% for testing

    # First, split into train + temp (val + test)
    train_idx, temp_idx, _, temp_labels = train_test_split(
        np.arange(len(dataset)),
        labels,
        stratify=labels,
        test_size=(1 - train_size),
        random_state=42,
    )

    # Then, split temp into validation and test
    val_idx, test_idx = train_test_split(
        temp_idx,
        stratify=temp_labels,
        test_size=(test_size / (test_size + val_size)),
        random_state=42,
    )

    # Create samplers
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    # Create DataLoaders
    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)
    # test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler)
    test_loader = DataLoader(dataset, sampler=test_sampler)
    return train_loader, val_loader, test_loader

def load_dataset(directory_path):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    dataset = torchvision.datasets.ImageFolder(root=directory_path, transform=transform)
    return dataset

In [14]:
def train_model(train_loader):
    net = CNN()
    print(net)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(10):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            print(i)
        print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
    return net


In [15]:
def test_model(net, test_loader, classes):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print(predicted, labels, correct)
    print(correct, total, correct / total * 100)

In [16]:
# train_loader, val_loader, test_loader = load_dataset(dir_path)
dataset = load_dataset('~/sgoinfre/leaves/images/Apple')
train_loader, val_loader, test_loader = create_dataloaders(dataset)


['Apple_Black_rot', 'Apple_healthy', 'Apple_rust', 'Apple_scab']


In [17]:
net = train_model(train_loader)


CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=59536, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=4, bias=True)
)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
[1,    70] loss: 0.041
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
[2,    70] loss: 0.032
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [18]:
test_model(net, test_loader, dataset.classes)

tensor([0]) tensor([0]) 1
tensor([0]) tensor([0]) 2
tensor([3]) tensor([2]) 2
tensor([1]) tensor([1]) 3
tensor([0]) tensor([0]) 4
tensor([1]) tensor([1]) 5
tensor([0]) tensor([0]) 6
tensor([3]) tensor([1]) 6
tensor([1]) tensor([1]) 7
tensor([3]) tensor([1]) 7
tensor([0]) tensor([0]) 8
tensor([1]) tensor([1]) 9
tensor([1]) tensor([1]) 10
tensor([3]) tensor([2]) 10
tensor([3]) tensor([3]) 11
tensor([1]) tensor([1]) 12
tensor([1]) tensor([1]) 13
tensor([1]) tensor([1]) 14
tensor([0]) tensor([0]) 15
tensor([1]) tensor([1]) 16
tensor([0]) tensor([1]) 16
tensor([1]) tensor([1]) 17
tensor([1]) tensor([1]) 18
tensor([1]) tensor([1]) 19
tensor([1]) tensor([1]) 20
tensor([0]) tensor([0]) 21
tensor([3]) tensor([1]) 21
tensor([3]) tensor([3]) 22
tensor([2]) tensor([0]) 22
tensor([3]) tensor([1]) 22
tensor([3]) tensor([3]) 23
tensor([1]) tensor([1]) 24
tensor([0]) tensor([0]) 25
tensor([1]) tensor([1]) 26
tensor([3]) tensor([3]) 27
tensor([1]) tensor([1]) 28
tensor([2]) tensor([2]) 29
tensor([1]) t